# Testing Differential Restore WITH STANDBY

I don't think I've actually done a differential restote with standby before!  So, instead of assume that it will work, and to document my success...

Goal: Restore successive Differential backups on a database over time while leaving the database available for read-only queries.

PRE-REQUISITE:  You need a local C:\\Temp folder or you need to change the scripts to read and write backup files to another location.

for [T-SQL command options on Restore, click here.](https://docs.microsoft.com/en-us/sql/t-sql/statements/restore-statements-transact-sql?view=sql-server-ver15)

## **We are going to first create a database...**

In [39]:
IF EXISTS (SELECT * FROM sys.databases WHERE name = 'DiffRestoreWithStandbyTest')
  DROP DATABASE [DiffRestoreWithStandbyTest]

Commands completed successfully.

Total execution time: 00:00:02.049

In [40]:
CREATE DATABASE [DiffRestoreWithStandbyTest]
GO

ALTER DATABASE [DiffRestoreWithStandbyTest] SET RECOVERY FULL
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.081

## Great, now lets create a table that logs the activity that coincides with the backup events.

In [41]:
CREATE TABLE [DiffRestoreWithStandbyTest].dbo.TranRecord 
    (
        [id] INT IDENTITY, 
        [Description] VARCHAR(100), 
        [TransactionTimeStamp] DATETIME DEFAULT GETDATE()
        )

Commands completed successfully.

Total execution time: 00:00:00.027

## Now, let's insert some data, then issue our 1st FULL backup...

In [42]:
INSERT INTO [DiffRestoreWithStandbyTest].dbo.TranRecord
  VALUES ('F1 - Full backup', GETDATE())
GO

BACKUP DATABASE [DiffRestoreWithStandbyTest] 
  TO DISK = 'C:\TEMP\F1.BAK'
  WITH INIT
GO

(1 row affected)

Processed 160 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 2 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

BACKUP DATABASE successfully processed 162 pages in 0.009 seconds (140.028 MB/sec).

Total execution time: 00:00:00.061

## Now, let's insert some _more_ data, then issue our 1st DIFFERENTIAL backup...

In [43]:
WAITFOR DELAY '00:00:15:00'

INSERT INTO [DiffRestoreWithStandbyTest].dbo.TranRecord
  VALUES ('D1 - Differential backup', GETDATE())
GO

BACKUP DATABASE [DiffRestoreWithStandbyTest] 
  TO DISK = 'C:\TEMP\D1.BAK' 
  WITH 
    INIT, 
    DIFFERENTIAL
GO

(1 row affected)

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

BACKUP DATABASE WITH DIFFERENTIAL successfully processed 65 pages in 0.007 seconds (71.707 MB/sec).

Total execution time: 00:00:15.064

## Now, let's insert _yet_ _more_ data, then issue our 2nd DIFFERENTIAL backup...

In [44]:
WAITFOR DELAY '00:00:15:00'

INSERT INTO [DiffRestoreWithStandbyTest].dbo.TranRecord
  VALUES ('D2 - Differential backup', GETDATE())
GO

BACKUP DATABASE [DiffRestoreWithStandbyTest] 
  TO DISK = 'C:\TEMP\D2.BAK' 
    WITH
      INIT,  
      DIFFERENTIAL
GO

(1 row affected)

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

BACKUP DATABASE WITH DIFFERENTIAL successfully processed 65 pages in 0.008 seconds (62.744 MB/sec).

Total execution time: 00:00:15.068

## The final stretch in setting things up...

## let's insert the last of our data, and issue the 3rd and last DIFFERENTIAL backup...

In [45]:
WAITFOR DELAY '00:00:15:00'

INSERT INTO [DiffRestoreWithStandbyTest].dbo.TranRecord
  VALUES ('D3 - Differential backup', GETDATE())
GO

BACKUP DATABASE [DiffRestoreWithStandbyTest] 
  TO DISK = 'C:\TEMP\D3.BAK' 
    WITH 
      INIT,
      DIFFERENTIAL
GO

(1 row affected)

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

BACKUP DATABASE WITH DIFFERENTIAL successfully processed 65 pages in 0.007 seconds (71.707 MB/sec).

Total execution time: 00:00:15.070

# **Excellent, now let's test the restore of the full backup WITH STANDBY**

In [46]:
RESTORE DATABASE [DiffRestoreWithStandbyTest] 
  FROM DISK = N'C:\Temp\F1.BAK' 
  WITH 
    REPLACE, 
    STANDBY = N'C:\Temp\DiffRestoreWithStandbyTest_RollbackUndoFile.tuf', 
    STATS = 25;
GO

29 percent processed.

54 percent processed.

79 percent processed.

100 percent processed.

Processed 160 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 2 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

RESTORE DATABASE successfully processed 162 pages in 0.014 seconds (90.018 MB/sec).

Total execution time: 00:00:00.167

## Okay, so let's see if we have read only access to the database!

In [47]:
SELECT * FROM [DiffRestoreWithStandbyTest].dbo.TranRecord
GO

(1 row affected)

Total execution time: 00:00:00.027

id,Description,TransactionTimeStamp
1,F1 - Full backup,2021-07-16 11:59:22.233


## Success - we can read from the database! 

## ... But lets see if we can restore our first differential and still have read-only access to the database

In [48]:
RESTORE DATABASE [DiffRestoreWithStandbyTest] 
  FROM DISK = N'C:\Temp\D1.BAK' 
  WITH 
    STANDBY = N'C:\Temp\DiffRestoreWithStandbyTest_RollbackUndoFile.tuf', 
    STATS = 25
GO

37 percent processed.

62 percent processed.

87 percent processed.

100 percent processed.

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

RESTORE DATABASE successfully processed 65 pages in 0.009 seconds (55.772 MB/sec).

Total execution time: 00:00:00.173

## Restored!  Now testing for read-only access...

In [49]:
SELECT * FROM [DiffRestoreWithStandbyTest].dbo.TranRecord
GO

(2 rows affected)

Total execution time: 00:00:00.028

id,Description,TransactionTimeStamp
1,F1 - Full backup,2021-07-16 11:59:22.233
2,D1 - Differential backup,2021-07-16 11:59:42.260


## Success!  Now to ensure that it is repeatable let's restore the 2nd differential backup, test...

In [50]:
RESTORE DATABASE [DiffRestoreWithStandbyTest] 
  FROM DISK = N'C:\Temp\D2.BAK' 
  WITH 
    STANDBY = N'C:\Temp\DiffRestoreWithStandbyTest_RollbackUndoFile.tuf', 
    STATS = 25
GO

37 percent processed.

62 percent processed.

87 percent processed.

100 percent processed.

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

RESTORE DATABASE successfully processed 65 pages in 0.011 seconds (45.632 MB/sec).

Total execution time: 00:00:00.163

In [51]:
SELECT * FROM [DiffRestoreWithStandbyTest].dbo.TranRecord
GO

(3 rows affected)

Total execution time: 00:00:00.028

id,Description,TransactionTimeStamp
1,F1 - Full backup,2021-07-16 11:59:22.233
2,D1 - Differential backup,2021-07-16 11:59:42.260
3,D2 - Differential backup,2021-07-16 12:00:03.103


## And the final stretch!

In [52]:
RESTORE DATABASE [DiffRestoreWithStandbyTest] 
  FROM DISK = N'C:\Temp\D3.BAK' 
  WITH 
    STANDBY = N'C:\Temp\DiffRestoreWithStandbyTest_RollbackUndoFile.tuf', 
    STATS = 25
GO

37 percent processed.

62 percent processed.

87 percent processed.

100 percent processed.

Processed 64 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest' on file 1.

Processed 1 pages for database 'DiffRestoreWithStandbyTest', file 'DiffRestoreWithStandbyTest_log' on file 1.

RESTORE DATABASE successfully processed 65 pages in 0.009 seconds (55.772 MB/sec).

Total execution time: 00:00:00.157

In [53]:
SELECT * FROM [DiffRestoreWithStandbyTest].dbo.TranRecord
GO

(4 rows affected)

Total execution time: 00:00:00.027

id,Description,TransactionTimeStamp
1,F1 - Full backup,2021-07-16 11:59:22.233
2,D1 - Differential backup,2021-07-16 11:59:42.260
3,D2 - Differential backup,2021-07-16 12:00:03.103
4,D3 - Differential backup,2021-07-16 12:00:20.363


So, looks like we can restore **with standby** to both database in a restoring mode, accepting of future differential (and transaction log) backups, while leaving the database in read-only mode thus allowing users continued access to the database!